<a href="https://colab.research.google.com/github/ysy9893/Colab_Tensorflow_Training/blob/main/train_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Object Detection Classifier on Colab 

**The original tutorial is for people using window os and gpu supported pc.**
**However, this tutorial referring most parts of original tutorial would support all os and who don't have any gpu supported pc in their households**





# 1. Downloads tensorflow Object Detection API repository from github and rename the filename to "tensorflow1"
url: https://github.com/tensorflow/models

Notice: When the runtime disconnects or halts, all downloaded folders would disappear. To cope with such situation, you have to move the folder into your google drive in advance.



In [ ]:
#!git clone https://github.com/tensorflow/models

# 2. Downloads the model from tensorflow's models and tutorial's repository 

There are a number of models provided by tensorflow, but in this case, we are gonna use the model mobilenet ssd version 2 quantized enabling fast inference in raspberrypi. 

You can download the model from the link below. 
https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf1_detection_zoo.md

You can download the tutorial's repository from the original tutorial's github (use the command git clone!)

https://github.com/EdjeElectronics/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10.git


# (Optional) Give some changes to object detection folder

If you would like to train the model using your own dataset, you have to make some changes to object detection folder. 

(Be noticed that you should not delete the folder)

Delete the following files 
1.   All files in \object_detection\images\train and
\object_detection\images\test

2.   The “test_labels.csv” and “train_labels.csv” files in \object_detection\images
3. All files in \object_detection\training
4. All files in \object_detection\inference_graph

# 3. Enable GPU runtime and confirm the connection of GPU with tensorflow 

In this tutorial the version of tensorflow is 1.15.2

In [1]:
%tensorflow_version 1.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print("tensorflow version: ",tf.__version__)

TensorFlow 1.x selected.
Found GPU at: /device:GPU:0
tensorflow version:  1.15.2


# 4. Install required packages 

In [ ]:
!python --version 

Python 3.7.10


In [2]:
#####################Packages needed by tensorflow#########################
!pip install protobuf #A language-neutral, platform-neutral, extensible way of serializing structured data for use in communications protocols, data storage, and more
!pip install pillow #Python Imaging Library
!pip install lxml #most feature-rich and easy-to-use library for processing XML and HTML in Python programming language
!pip install Cython #source code translator based on Pyrex, but supports more cutting edge functionality and optimizations
!pip install contextlib2
!pip install jupyter
!pip install matplotlib #library for creating static, animated, and interactive visualizations in Python
!pip install tf_slim 
!pip install pycocotools
####################Packages needed to generate TFRecords and to manipulate output images####################
!pip install pandas #open source data analysis and manipulation tool, built on top of the Python programming language
!pip install opencv-python #OpenCV-Python is a library of Python bindings designed to solve computer vision problems.

     |████████████████████████████████| 358kB 5.9MB/s 


# 5. Configure PYTHONPATH environment variable 

PYTHONPATH variable must be created to the points at 


1. tensorflow1/models/research
2. tensorflow1/models/research/slim
3. tensorflow1/models

Doing this, you can utilize python modules from those directories. 


In [3]:
!set PYTHONPATH=/content/drive/MyDrive/인턴프로젝트/tensorflow1/models/research/slim

# 6. Compile Protobuf and run setup.py

Compiling protobuf files enables configuration of model and training parameters. 

In [4]:
%cd /content/drive/MyDrive/인턴프로젝트/tensorflow1/models/research

/content/drive/MyDrive/인턴프로젝트/tensorflow1/models/research


By implementing the command below, a name_pb2.py for each name.proto file would be newly created. 

In [ ]:
%%bash
protoc object_detection/protos/*.proto --python_out=.

Now, run setup.py from /content/drive/MyDrive/인턴프로젝트/tensorflow1/models/research/slim

Notice: It is advised to delete BUILD file before running setup.py

In [5]:
%cd /content/drive/MyDrive/인턴프로젝트/tensorflow1/models/research/slim

/content/drive/MyDrive/인턴프로젝트/tensorflow1/models/research/slim


In [6]:
!python setup.py build 
!python setup.py install 

running build
running build_py
running egg_info
writing slim.egg-info/PKG-INFO
writing dependency_links to slim.egg-info/dependency_links.txt
writing requirements to slim.egg-info/requires.txt
writing top-level names to slim.egg-info/top_level.txt
writing manifest file 'slim.egg-info/SOURCES.txt'
running install
running bdist_egg
running egg_info
writing slim.egg-info/PKG-INFO
writing dependency_links to slim.egg-info/dependency_links.txt
writing requirements to slim.egg-info/requires.txt
writing top-level names to slim.egg-info/top_level.txt
writing manifest file 'slim.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/preprocessing
copying build/lib/preprocessing/__init__.py -> build/bdist.linux-x86_64/egg/preprocessing
copying build/lib/preprocessing/inception_preprocessing.py -> build/bdist.linux-x86_64/egg/preprocessing
copying build/lib/prepr

# 7. Collect and label data
Collect your own data for training object detection calssifier. There's so many website providing well organized dataset!!(ex.Kaggle, OpenImage)

Once you created dataset, it's time to label all desired objects for object detection using labeling software. 
I recommend you to use labelImg and set the label format to .xml!!

labelImg URL: https://github.com/tzutalin/labelImg



# 8. Generate TFRecord 

You cannot use .xml format files to train tensorflow model. To train tensorflow model, you need TFRecord serving as input feed, and there is a intermediate step to convert all xml files to TFRecord. 



*8.1 Convert xml files to csv files*

In [7]:
%cd /content/drive/MyDrive/인턴프로젝트/tensorflow1/models/research/object_detection 

/content/drive/MyDrive/인턴프로젝트/tensorflow1/models/research/object_detection


In [8]:
!python xml_to_csv.py

Successfully converted xml to csv.
Successfully converted xml to csv.


*8.2 Convert csv to TFRecord*

Before running the script generating tf_record, edit some of lines in the script.

Find the function defining class_text_to_int and edit names of classes according to your dataset classes.

(Notice: If there are tfrecord files previously created, you should delete them !!!) 

In [9]:
%cd /content/drive/MyDrive/인턴프로젝트/tensorflow1/models/research/object_detection

/content/drive/MyDrive/인턴프로젝트/tensorflow1/models/research/object_detection


In [10]:
!python generate_tfrecord.py --csv_input=./images/train_labels.csv --image_dir=./images/train --output_path=train.record



W0405 02:27:41.897979 140388544071552 module_wrapper.py:139] From generate_tfrecord.py:95: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W0405 02:27:42.158507 140388544071552 module_wrapper.py:139] From generate_tfrecord.py:54: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Successfully created the TFRecords: /content/drive/MyDrive/인턴프로젝트/tensorflow1/models/research/object_detection/train.record


No object_detection module error would be addressed replacing "from object_detection.utils" with "from utils"

In [11]:
!python generate_tfrecord.py --csv_input=./images/test_labels.csv --image_dir=./images/test --output_path=test.record



W0405 02:41:16.945347 140661296256896 module_wrapper.py:139] From generate_tfrecord.py:95: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W0405 02:41:17.055204 140661296256896 module_wrapper.py:139] From generate_tfrecord.py:54: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Successfully created the TFRecords: /content/drive/My Drive/인턴프로젝트/tensorflow1/models/research/object_detection/test.record


# 9. Create label map and configure training 

This is the last preparation or configuration right before training the model.

1. Label Map : Mapping each class(category) to index


Make a new file under /object_detection/trainig direcotry and name it labelmap.pbtxt
The format of the content should look like this. 

  item {
  id: 1
  name: 'class1'
}

  item {
  id: 2
  name: 'class2'
}

  item {
  id: 3
  name: 'class3'
}


2. Change config file

Let's look up the config file of model used for running training from /object_detection/samples/configs directory. 

In this tutorial mobilent_ssd_v2_quantized would be used. 

Copy and paste the file into the /object_detection/trainig directory. 

Open it using the text editor and follow instructions below. 

* Line 9. Change num_classes to the number of different objects you want the classifier to detect. For my bird/squirrel/raccoon detector example, there are three classes, so I set num_classes: 3

* Line 141. Change batch_size: 24 to batch_size: 6 . The smaller batch size will prevent OOM (Out of Memory) errors during training.

* Line 156. Change fine_tune_checkpoint to: "C:/tensorflow1/models/research/object_detection/ ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03/model.ckpt."

* Line 175. Change input_path to: "C:/tensorflow1/models/research/object_detection/train.record"

* Line 177. Change label_map_path to: "C:/tensorflow1/models/research/object_detection/training/labelmap.pbtxt"

* Line 181. Change num_examples to the number of images you have in the \images\test directory. For my bird/squirrel/raccoon detector example, there are 582 test images, so I set num_examples: 582.

* Line 189. Change input_path to: "C:/tensorflow1/models/research/object_detection/test.record"

* Line 191. Change label_map_path to: "C:/tensorflow1/models/research/object_detection/training/labelmap.pbtxt"

# 10. Run training 



In [12]:
%cd /content/drive/MyDrive/인턴프로젝트/tensorflow1/models/research
!pip install lvis
!pip uninstall nets

/content/drive/MyDrive/인턴프로젝트/tensorflow1/models/research


In [ ]:
!python model_main.py --model_dir=./object_detection/training/ --pipeline_config_path=./object_detection/training/ssd_mobilenet_v2_quantized_300x300_coco.config

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
INFO:tensorflow:global_step/sec: 5.27347
I0405 04:18:31.038236 139725487048576 basic_session_run_hooks.py:692] global_step/sec: 5.27347
INFO:tensorflow:loss = 4.1233797, step = 26200 (18.963 sec)
I0405 04:18:31.039295 139725487048576 basic_session_run_hooks.py:260] loss = 4.1233797, step = 26200 (18.963 sec)
INFO:tensorflow:global_step/sec: 5.27652
I0405 04:18:49.990117 139725487048576 basic_session_run_hooks.py:692] global_step/sec: 5.27652
INFO:tensorflow:loss = 4.0196834, step = 26300 (18.952 sec)
I0405 04:18:49.991282 139725487048576 basic_session_run_hooks.py:260] loss = 4.0196834, step = 26300 (18.952 sec)
INFO:tensorflow:global_step/sec: 5.26497
I0405 04:19:08.983584 139725487048576 basic_session_run_hooks.py:692] global_step/sec: 5.26497
INFO:tensorflow:loss = 2.1816435, step = 26400 (18.994 sec)
I0405 04:19:08.984798 139725487048576 basic_session_run_hooks.py:260] loss = 2.1816435, step = 26400 (18.994 sec)
INFO:tensorflow:global_step/sec: 5

# 11. Export the inference graph 

This create the .pb file under /object_detection/inference_graph directory.

NOTICE: Unless you have to copy and move export_inference_graph.py into /research,there would be object detection module not found error.

In [ ]:
%cd /content/drive/MyDrive/인턴프로젝트/tensorflow1/models/research

/content/drive/MyDrive/인턴프로젝트/tensorflow1/models/research


In [ ]:
!python export_tflite_ssd_graph.py --input_type image_tensor --pipeline_config_path=./object_detection/training/ssd_mobilenet_v2_quantized_300x300_coco.config --trained_checkpoint_prefix ./object_detection/training/model.ckpt-200000 --output_directory ./object_detection/inference_graph/new_model --add_postprocessing_op=true

Instructions for updating:
Please use `layer.__call__` method instead.
W0404 16:50:32.873378 140638971271040 deprecation.py:323] From /usr/local/lib/python3.7/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0404 16:50:35.583910 140638971271040 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0404 16:50:35.620477 140638971271040 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0404 16:50:35.657673 140638971271040 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

In [ ]:
%cd /content/drive/MyDrive/인턴프로젝트/tensorflow1/models/research/object_detection 

/content/drive/MyDrive/인턴프로젝트/tensorflow1/models/research/object_detection


In [ ]:
%reload_ext tensorboard

In [ ]:
!tensorboard --logdir=./training 

2021-03-30 06:47:32.310710: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
W0330 06:47:33.938837 139881594504960 plugin_event_accumulator.py:322] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0330 06:47:33.983711 139881594504960 plugin_event_accumulator.py:334] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0330 06:47:35.330381 139881594504960 plugin_event_accumulator.py:322] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0330 06:47:35.388519 139881594504960 plugin_event_accumulator.py:334] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0330 06:47:35.455139 13988159450496